In [6]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

def make_dataloader(n):
    X = np.random.random((n, 2)) * 2 - 1
    Y = (X[:, 0] > X[:, 1] + 0.25).astype(int) + 1

    X = torch.tensor(X, dtype=torch.float)
    Y = torch.tensor(Y, dtype=torch.long)

    div1 = int(n*0.8)
    div2 = int(n*0.9)
    Xs = [X[:div1], X[div1:div2], X[div2:]]
    Ys = [Y[:div1], Y[div1:div2], Y[div2:]]

    dataset = TensorDataset(Xs[0], Ys[0])
    data_loader = DataLoader(dataset, batch_size=4)
    return data_loader

In [8]:
import torch.nn as nn
from metal.mmtl.task import Task
from BERT_tasks import create_task

foo_input = nn.Linear(2, 10)
bar_input = foo_input #nn.Linear(100, 7)

foo_head = nn.Linear(10, 2)
bar_head = nn.Linear(10, 2)

foo_data = make_dataloader(5000)
bar_data = make_dataloader(2000)

foo = Task("foo_task", {"train":foo_data, "valid":foo_data, "test":None}, foo_input, foo_head, )
bar = create_task('SST2')
baz = create_task('COLA')

Loading SST2 Dataset




  0%|          | 0/67349 [00:00<?, ?it/s]

  0%|          | 239/67349 [00:00<00:28, 2387.65it/s]

  1%|          | 628/67349 [00:00<00:24, 2699.97it/s]

  1%|▏         | 972/67349 [00:00<00:22, 2886.18it/s]

  2%|▏         | 1244/67349 [00:00<00:23, 2829.30it/s]

  2%|▏         | 1659/67349 [00:00<00:21, 3127.42it/s]

  3%|▎         | 2107/67349 [00:00<00:18, 3437.78it/s]

  4%|▍         | 2537/67349 [00:00<00:17, 3655.68it/s]

  4%|▍         | 2991/67349 [00:00<00:16, 3879.02it/s]

  5%|▌         | 3418/67349 [00:00<00:16, 3987.99it/s]

  6%|▌         | 3857/67349 [00:01<00:15, 4100.15it/s]

  6%|▋         | 4304/67349 [00:01<00:15, 4202.06it/s]

  7%|▋         | 4822/67349 [00:01<00:14, 4453.11it/s]

  8%|▊         | 5391/67349 [00:01<00:13, 4763.27it/s]

  9%|▉         | 5978/67349 [00:01<00:12, 5048.14it/s]

 10%|▉         | 6566/67349 [00:01<00:11, 5271.27it/s]

 11%|█         | 7169/67349 [00:01<00:10, 5477.10it/s]

 12%|█▏        | 7758/67349 [00:01<00:10, 5594.32it/s]

 12%|█

TypeError: super(type, obj): obj must be an instance or subtype of type

In [10]:
from metal.end_model import EndModel
from metal.mmtl.metal_model import MetalModel
from metal.mmtl.trainer import MultitaskTrainer

tasks = [bar,baz]
model = MetalModel(tasks, verbose=False)
trainer = MultitaskTrainer()
trainer.train_model(
    model, 
    tasks, 
    n_epochs=3, 
    lr=0.005, 
    progress_bar=True,
    log_every=0.2,
    log_unit="epochs",
    score_every=0.4,
    checkpoint_best=True,
    checkpoint_metric="SST2/accuracy",
    checkpoint_metric_mode="max"
)

Beginning train loop.
Expecting a total of 75900 examples and 75140 batches per epoch from 2 tasks.


KeyboardInterrupt: 